# 2  Решающие деревья

Решающие деревья (descision treeы)


#### 2.1.4 Какого музыканта Beatles я загадал или entropy reduction

Небольшая ремарка о том, как получить представление данных в виде точек в пространстве.

Представьте, что у вас есть маленький датафрэйм с 2-мя характеристиками цветов и их видом (который мы потом будем предсказывать, то есть это целевая переменная)

petal_length    petal_width    species
1.4             0.2            0
4.5             1.5            1
5.1             1.9            2

Теперь просто возьмём каждое наблюдение из датасэта и поместим его в пространство, где будет 2 оси, соответствующие фичам - petal length и petal width. 1 точка в пространстве - 1 наблюдение из датафрэйма, у которого положение в пространстве определяется значениями petal length и petal width. Получится следующая картина (цвета соответствуют виду цветка)


Такая репрезентация бывает очень удобной для представления работы моделей. Точно так же строится и в случае большего числа фичей - просто увеличивается число осей и пространство становится многомерным.

В случае определения являются ли письма спамом или нет, исходные данные представляют собой текст. Его так просто не запихнёшь в большинство моделей и для представления датасэта как множества точек нужны предварительные преобразования. Как вариант - посчитать число букв и число восклицательных знаков в каждом письме и уже на основании этих данных произвести визуализацию

Комментарии

Anonymous 53246858
Плохо представляю себе многомерное пространство в случае большого кол-ва фичей. Можно пример?


Филипп Филиппак
@Anonymous_53246858, можно рассмотреть это так. У вас есть n-мерное пространство фичей, а каждый вопрос решающего дерева — это подпространство мерности n-1. То есть: если фича одна, то её значения — на прямой (одномерное пространство), а разделяющее подпространство — точка (нульмерное пространство). Дальше, если две фичи, то наблюдения лежат на плоскости, а разделяющее подпространство — одномерное — это прямая.

Вообще, такое разделяющее пространство мерности n-1 зовётся гиперплоскостью. Итак, для трёхмерного пространства фичей (например, мы можем классифицировать что-то по цвету, размеру и весу) гиперплоскостью будет плоскость. Четырёхмерное пространство сложно представить визуально, но концепция та же: есть пространство с четырьмя измерениями, а трёхмерная гиперплоскость его делит. И так далее для больших размерностей.

Могу порекомендовать вот это — http://hi.gher.space/classic/introduction.htm — чтение для лучшего понимания многомерных пространств в общем случае.

In [ ]:
# 2.1.6 Какого музыканта Beatles я загадал или entropy reduction


In [1]:
from sklearn import tree
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#data = pd.read_csv('example_data.csv')
data = pd.DataFrame({'X_1': [1, 1, 1, 0, 0, 0, 0, 1], 'X_2': [0, 0, 0, 1, 0, 0, 0, 1], 'Y': [1, 1, 1, 1, 0, 0, 0, 0]})


In [3]:
data

,X_1,X_2,Y
0,1,0,1
1,1,0,1
2,1,0,1
3,0,1,1
4,0,0,0
5,0,0,0
6,0,0,0
7,1,1,0


Задача: научиться на основании переменных X_1, X_2 предсказывать значение Y

In [4]:
clf = tree.DecisionTreeClassifier()

In [5]:
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
#
X = data[['X_1', 'X_2']]
# 
y = data.Y

In [7]:
# Обучение дерева решений
clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Теперь попробуем вытащить знания, которые сформировало дерево решений.
Самый простой способ понять - это визуализировать дерево

In [8]:
from IPython.display import SVG
from graphviz import Source
from IPython.display import display

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
from IPython.display import HTML
style = "<style>svg{width:70% !important; height: 70% !important;}/<style>"
HTML(style)

#### entropy reduction и Information gain

1)
$X_2 > \frac{1}{2}$

(-) xxx ooo
(+) x   o

2)
$X_1 > \frac{1}{2}$

(-) x ooo
(+) xxx o


IG (information gain) - 

$$ E = - \sum\limits_{i=1}^n p_i \cdot log_2(p_i) $$

$ X\_1 > \frac{1}{2}$ 

$E = - \frac{1}{4} \cdot log_2(\frac{1}{4}) - \frac{3}{4} \cdot log_2(\frac{3}{4})$

In [10]:
import math

In [11]:
print(- 0.25 * math.log(0.25, 2))
print(- 0.75 * math.log(0.75, 2))
print(- 0.25 * math.log(0.25, 2) - 0.75 * math.log(0.75, 2))

0.5
0.31127812445913283
0.8112781244591328


##### information gain 

Сколько информации мы выиграли. Это исходная полная энтропия E(Y) минус условная энтропия E(Y/X). Т.е. что у нас произошло с неопределенностью, после того, как мы учли значения некоторой переменной X.

IG = E(Y) - E(Y/X)

Сколько наблюдений было всего N и сколько получилось в выборках n_1 и n_2 после сплита. 

$E(Y/X) = \frac{n_1}{N} \cdot E_1 + \frac{n_2}{N} \cdot E_2)$


Посчитаем, чему равен information gain в первом случае

$IG = 1 - (\frac{6}{8} \cdot 1 + \frac{2}{8} \cdot 1) = 0$

Посчитаем IG для второго примера

$IG = 1 - (\frac{4}{8} \cdot 0.81 + \frac{4}{8} \cdot 0.81) = 1.19$

Таким образом, делая разбитие по X_1 мы получи больше информации.

После разбития X_1 lелаем следующий сплит


($X\_2 > \frac{1}{2}$)(-)
(-) x
(+) ooo 

($X\_2 > \frac{1}{2}$)(-)
(-)
(+)


#### 2.2.6 

Представьте, что вы решили научить робота для животных отличать собачек от котиков. Для начала проанализируем данные - отметьте все верные утверждения о данном датафрэйме


In [12]:
import pandas as pd

In [13]:
dogs = pd.read_csv('https://stepik.org/media/attachments/course/4852/dogs.csv')
dogs

,Unnamed: 0,Шерстист,Гавкает,Лазает по деревьям,Вид
0,0,1,1,0,собачка
1,1,1,1,0,собачка
2,2,1,1,0,собачка
3,3,1,1,0,собачка
4,4,1,0,1,котик
5,5,1,0,1,котик
6,6,1,0,1,котик
7,7,1,0,1,котик


#### 2.2.7

В нашем Big Data датасэте появились новые наблюдения! Давайте немного посчитаем энтропию, чтобы лучше понять, формализуемость разделения на группы.

Формат записи - энтропия в группе, где переменная равна 0 и энтропия в группе, где переменная равна 1 (десятичный разделитель - точка, округляйте до 2-ого знака при необходимости).


In [14]:
import pandas as pd
import math
import numpy as np

In [15]:
url = 'https://stepik.org/media/attachments/course/4852/cats.csv'
cats = pd.read_csv(url, index_col = 0)
cats.columns = [x.replace(" ", "_") for x in cats.columns]
cats

,Шерстист,Гавкает,Лазает_по_деревьям,Вид
0,1,1,0,собачка
1,1,1,0,собачка
2,1,1,0,собачка
3,1,1,0,собачка
4,1,0,1,котик
5,1,0,1,котик
6,1,0,1,котик
7,1,0,1,котик
8,1,1,1,котик
9,0,0,1,котик


In [16]:
df = cats
df

,Шерстист,Гавкает,Лазает_по_деревьям,Вид
0,1,1,0,собачка
1,1,1,0,собачка
2,1,1,0,собачка
3,1,1,0,собачка
4,1,0,1,котик
5,1,0,1,котик
6,1,0,1,котик
7,1,0,1,котик
8,1,1,1,котик
9,0,0,1,котик


In [17]:
np.log2(0.4444444444444444)

-1.1699250014423124

In [18]:
math.log(0.4444444444444444, 2)

-1.1699250014423124

In [19]:
def calc_entropy(df = pd.DataFrame(), x = '', y = ''):
    y_unique = df[y].unique().tolist()    # Уникальные значения столбца y   
    x_unique = df[x].unique().tolist()    # Уникальные значения столбца x
    tot_cnt = len(df)
    
    E = 0
    IG = 0 
    for y_it in y_unique:
        print('y_it: ', y_it)
        sum_e = 0
        for x_it in x_unique:
            print('x_it: ', x_it)
            x_it_cnt = len(df.loc[(df[x] == x_it)])        # Количество записей, где x = x_it
            p = len(df.loc[(df[y] == y_it) & (df[x] == x_it)]) / x_it_cnt   # Вероятность исхода y_it при значении x = x_it
            print('p: ', p)
            if p != 0:
                print('p * np.log2(p)', p * np.log2(p))
                sum_e = sum_e + p * np.log2(p)
                print('it_e: ', sum_e)
        print('sum_e =', sum_e)
        E = E - sum_e
        IG = IG + x_it_cnt / tot_cnt * sum_e
        print('sum_IG = ', IG)
    print('E = ', E)
    print('IG = ', 1 - IG)
    
    



In [20]:
calc_entropy(df, x = 'Шерстист', y = 'Вид')

y_it:  собачка
x_it:  1
p:  0.4444444444444444
p * np.log2(p) -0.5199666673076944
it_e:  -0.5199666673076944
x_it:  0
p:  0.0
sum_e = -0.5199666673076944
sum_IG =  -0.05199666673076944
y_it:  котик
x_it:  1
p:  0.5555555555555556
p * np.log2(p) -0.4711093925305278
it_e:  -0.4711093925305278
x_it:  0
p:  1.0
p * np.log2(p) 0.0
it_e:  -0.4711093925305278
sum_e = -0.4711093925305278
sum_IG =  -0.09910760598382223
E =  0.9910760598382222
IG =  1.0991076059838223


In [21]:
calc_entropy(df, x = 'Гавкает', y = 'Вид')

y_it:  собачка
x_it:  1
p:  0.8
p * np.log2(p) -0.2575424759098898
it_e:  -0.2575424759098898
x_it:  0
p:  0.0
sum_e = -0.2575424759098898
sum_IG =  -0.1287712379549449
y_it:  котик
x_it:  1
p:  0.2
p * np.log2(p) -0.46438561897747244
it_e:  -0.46438561897747244
x_it:  0
p:  1.0
p * np.log2(p) 0.0
it_e:  -0.46438561897747244
sum_e = -0.46438561897747244
sum_IG =  -0.36096404744368116
E =  0.7219280948873623
IG =  1.360964047443681


In [22]:
calc_entropy(df, x = 'Лазает_по_деревьям', y = 'Вид')

y_it:  собачка
x_it:  0
p:  1.0
p * np.log2(p) 0.0
it_e:  0.0
x_it:  1
p:  0.0
sum_e = 0.0
sum_IG =  0.0
y_it:  котик
x_it:  0
p:  0.0
x_it:  1
p:  1.0
p * np.log2(p) 0.0
it_e:  0.0
sum_e = 0.0
sum_IG =  0.0
E =  0.0
IG =  1.0


In [23]:
x = 'Шерстист'
y = 'Вид'
tot_cnt = len(df)
y_it = 1

In [24]:
#df[x] == 0
#len(df[x])
#df.loc[df[x] == y_it]
len(df.loc[df[x] == y_it]) / tot_cnt 

0.9

In [31]:
import math
import numpy as np

In [26]:
#Энтропия при разделении по фиче Шерстист в группах, где Шерстист равно 0 и 1 соответственно, составляет

# шерстист = 0
# наблюдений 
N = 1 
n_1 = 1
n_2 = 0

E = 0
#E = - n_1 / N * math.log(n_1 / N, 2) - n_2 / N * math.log(n_2 / N, 2)
print('шерстист = 0. E = ', E)

# шерстист = 1
# наблюдений 
N = 9
n_1 = 4
n_2 = 5

E = - n_1 / N * math.log(n_1 / N, 2) - n_2 / N * math.log(n_2 / N, 2)
print('шерстист = 1. E = ', E)

шерстист = 0. E =  0
шерстист = 1. E =  0.9910760598382222


In [44]:
#Энтропия при разделении по фиче Гавкает в группах, где Гавкает равно 0 и 1 соответственно, составляет
# Гавкает = 0
# наблюдений 
N = 5 
n_1 = 0
n_2 = 5

E = 0
#E = - n_1 / N * math.log(n_1 / N, 2) - n_2 / N * math.log(n_2 / N, 2)
print('Гавкает = 0. E = ', E)

# Гавкает = 1
# наблюдений 
N = 5
n_1 = 4
n_2 = 1

E = - n_1 / N * math.log(n_1 / N, 2) - n_2 / N * math.log(n_2 / N, 2)
print('Гавкает = 1. E = ', E)


Гавкает = 0. E =  0
Гавкает = 1. E =  0.7219280948873623


In [29]:
# Энтропия при разделении по фиче Лазает по деревьям в группах, где эта фича равна 0 и 1 соответственно, составляе
# Лазает по деревьям = 0
# наблюдений 
N = 4 
n_1 = 0
n_2 = 4

E = 0
E = - n_1 / N * math.log(n_1 / N, 2) - n_2 / N * math.log(n_2 / N, 2)
print('Лазает по деревьям = 0. E = ', E)

# Лазает по деревьям = 1
# наблюдений 
N = 6
n_1 = 6
n_2 = 0

E = - n_1 / N * math.log(n_1 / N, 2) - n_2 / N * math.log(n_2 / N, 2)
print('Лазает по деревьям = 1. E = ', E)

#----
import numpy as np
np.log2(x)

import math
math(x, 2)

#Одна кошка гавкает

ValueError: math domain error

#### 2.2.8

Ещё немного арифметики - посчитаем Information Gain по данным из предыдущего задания. Впишите через пробел округлённые до 2-ого знака значения IG для фичей Шерстист, Гавкает и Лазает по деревьям. Десятичным разделителем в данном задании является точка.

По какой переменной нужно делать сплит, исходя из всего этого (не вводите ответ на этот вопрос в поле, просто подумайте)


In [37]:
E_Y = -0.4 * np.log2(0.4) -0.6 * np.log2(0.6)
print('E_Y: ', E_Y)

E_Y:  0.9709505944546686


In [ ]:
1.1 1.36 0.97

In [47]:
print('E_Y: ', E_Y)

# Шерстист
print('Шерстист ', E_Y - (0.9910760598382222 * 0.9))

# Гавкает
print('Гавкает ', E_Y - (0.7219280948873623 * 0.5))

# Лазит по деревьям
print('Лазит по деревьям ', E_Y - 0)

E_Y:  0.9709505944546686
Шерстист  0.07898214060026854
Гавкает  0.6099865470109874
Лазит по деревьям  0.9709505944546686


### 2.3 Titanic: Machine Learning from Disaster


In [1]:
print('test')

test


In [2]:
from sklearn import tree
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from IPython.display import SVG
from graphviz import Source
from IPython.display import display

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
from IPython.display import HTML
style = "<style>svg{width:70% !important; height:70% !important;}</style>"
HTML(style)

In [ ]:
url_tit = r'g:\work\!data\kaggle_titanic\train.csv'

In [ ]:
titanic_data = pd.read_csv(url_tit)